#### 1. Loading Variables & render_md

In [40]:
from dotenv import load_dotenv
load_dotenv()
from IPython.display import display,Markdown
def render_md(text:str):
    return display(Markdown(text))


#### 2. Creating Agent

> ## @Creating_Tools

#### 1-search tool

In [41]:
from langchain_tavily import TavilySearch
from langchain.tools import tool
@tool("search_tool")
def search_tool(query:str):
    """This Tool is use for search through internet and get results"""
    tool = TavilySearch(
        max_results=5,
        topic="general",
    )
    tool_result = tool.invoke(query)
    return tool_result

#### 2-contact detail tool

In [83]:
contacts = {
    "ash":{
        "email":"muth@muthal.in",
        "phone":143
    },
    "adarsh":{
        "email":"adarsh@google.com",
        "phone":9090808060
    },
    "shreya":{
        "email":"shreya@google.com",
        "phone":91235458
    },
    "serena":{
        "email":"serena@google.com",
        "phone":21354889
    }

}

@tool
def get_contact(name:str):
    """Get Contact Info(such as email & phone no.) of people"""
    if not name in contacts:
        return "Contact Not Found"
    print(name)
    return contacts[name]
    
    

In [84]:
"l" if "shreya" in contacts else "5"

'l'

#### 3-weather tool

In [43]:
import os
import requests
from langchain.tools import tool

@tool
def get_weather_tool(city_name: str, aqi: bool = False):
    """
    Get weather details and (optional) Air Quality Index for a city.
    Includes timeout to avoid hanging requests.
    """

    api_key = os.getenv("WEATHER_API_KEY")
    if not api_key:
        raise ValueError("No API KEY was Found!")

    base_url = "https://api.weatherapi.com/v1/current.json"

    params = {
        "key": api_key,
        "q": city_name,
        "aqi": "yes" if aqi else "no"
    }

    print(f"Requesting weather for: {city_name}...")

    try:
        # ⏱ timeout=(connect_timeout, read_timeout)
        response = requests.get(
            base_url,
            params=params,
            timeout=(3, 5)  # 3s to connect, 5s to receive data
        )

        if response.status_code == 200:
            return response.json()
        else:
            return f"Error {response.status_code}: {response.text}"

    except requests.exceptions.Timeout:
        return "Error: Request timed out. Please try again later."

    except requests.exceptions.RequestException as e:
        return f"Network error occurred: {e}"


#### Tool_list

In [85]:
tool_list = [get_contact,get_weather_tool,search_tool]

#### Making Agent

In [86]:
from langchain.agents import create_agent
agent = create_agent(
    model="groq:qwen/qwen3-32b",
    tools=tool_list
)

> Play with the prompt here to check the working of agent !

In [116]:
prompt = {
    "messages":[{
        "role":"user",
        "content":"what is current exchange rate of usd to inr ?"
    }]
}

In [117]:
res = agent.invoke(prompt)

In [118]:
res

{'messages': [HumanMessage(content='what is current exchange rate of usd to inr ?', additional_kwargs={}, response_metadata={}, id='b4d968ca-daa5-47f0-b401-2a5951decf79'),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the current exchange rate from USD to INR. I need to find out the latest rate. Let me check which tool I can use here. The available tools are get_contact, get_weather_tool, and search_tool.\n\nThe search_tool is designed to search the internet and get results. Since exchange rates can change frequently, the best way to get the current rate is to perform a quick search. The other tools don\'t seem relevant here. So I\'ll use the search_tool with a query like "current USD to INR exchange rate". That should fetch the latest information from the web.\n', 'tool_calls': [{'id': 'dxdg9pad4', 'function': {'arguments': '{"query":"current USD to INR exchange rate"}', 'name': 'search_tool'}, 'type': 'function'}]}, response_metadata={'

In [119]:
render_md(res['messages'][-1].content)

The current exchange rate for 1 US Dollar (USD) to Indian Rupees (INR) is approximately **₹90.20** based on recent data from multiple sources. However, rates may vary slightly depending on the service provider (e.g., banks, currency converters, or money transfer platforms) due to fees or market fluctuations. For the most accurate and up-to-date rate, you can check platforms like [Xe](https://www.xe.com) or [Wise](https://wise.com).